# Opening Lead

This is a tutorial for how to use the opening lead engine.

In [1]:
import os
import sys
sys.path.append('../../src')
os.environ['BEN_HOME'] = "../.."

from nn.models import Models
from bots import BotLead
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
models = Models.from_conf(conf.load('../Conf/UCBC2024.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('../Conf/UCBC2024.conf'), False)  # Load sampling strategies

INFO:tensorflow:Restoring parameters from ..\Models/bidding-3233000
INFO:tensorflow:Restoring parameters from ..\Models/binfo-808000
INFO:tensorflow:Restoring parameters from ..\Models/lead_suit-154000
INFO:tensorflow:Restoring parameters from ..\Models/lead_nt-59000
INFO:tensorflow:Restoring parameters from ..\Models/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\Models/lefty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/dummy-920000
INFO:tensorflow:Restoring parameters from ..\Models/righty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/decl-1000000


In [3]:
# both vulnerable. you are sitting North as dealer and you hold
hand = '5.A97643.Q82.QJ5'

# the auction goes:
auction = ["PAD_START", "PAD_START", "PAD_START", "1C", "PASS", "2S", "PASS", "3S", "PASS", "4D",  "PASS", "4S", "PASS", "4N", "PASS", "5D", "PASS", "6S", "PASS", "PASS", "PASS"]

# what to lead?

lead_bot = BotLead([True, True], hand, models, sampler, False)
lead = lead_bot.find_opening_lead(auction)

In [4]:
lead.card

HA

seems like the engine chose to lead the ace of diamonds

the other options it considered were: a small spade and a small club

In [5]:
lead.to_dict()['candidates']

[{'card': 'HA',
  'insta_score': 0.3214,
  'expected_tricks_sd': 11.422,
  'p_make_contract': 0.516},
 {'card': 'CQ',
  'insta_score': 0.3945,
  'expected_tricks_sd': 11.433,
  'p_make_contract': 0.528},
 {'card': 'Dx',
  'insta_score': 0.2199,
  'expected_tricks_sd': 11.52,
  'p_make_contract': 0.565}]

in the above output:
- `insta_score` reflects the preference of the neural network
- `expected_tricks` how many tricks declarer is expected to take on this lead
- `p_make_contract` is the probability of the contract making on this lead

the probability of making and the expected tricks are computed on samples which are consistent with the auction. the samples are estimated single dummy using a neural network (more on this in another tutorial). we could also solve the samples double dummy, but that would be quite a bit slower.

In [6]:
# each row is one sample board
# the hands are in the order: LHO, Partner, RHO. Your cards are not shown as they are fixed/

lead.samples

['98x.T.Kxx.AKT9xx xxx.Kx.JT9xxx.8x AKQJTx.QJ8x.A.xx 0.38323',
 'xxxx.Kx.x.AK9xxx JT9.QJ8.JT9xx.8x AKQ8x.Tx.AKxx.Tx 0.37654',
 '9xx.KJ.Kx.AT9xxx T8x.Tx.JT9xxx.Kx AKQJxx.Q8x.Ax.8x 0.35281',
 'T9xx.Qx.KJx.AK98 8x.8x.T9xxx.xxxx AKQJxx.KJT.Ax.Tx 0.33033',
 '9xx.JT.Kxx.AKTxx T8.8xx.ATxxx.98x AKQJxxx.KQ.J9.xx 0.30402',
 'Txx.KT.J9x.AK9xx J8x.Q8x.Txxx.Txx AKQ9xx.Jx.AKx.8x 0.30138',
 'KJx.T.Kxxx.Kxxxx T9xx.xx.JT9xx.Ax AQ8xx.KQJ8.A.T98 0.28540',
 'J8x.KTx.Tx.AK8xx xxx.QJ.9xxxx.9xx AKQT9x.8x.AKJ.Tx 0.28250',
 '8xx.J.K9x.AKTxxx xx.T8x.JTxxxx.8x AKQJT9x.KQx.A.9x 0.28136',
 'T9x.K8xx.Jx.AKT9 Jxx.T.KT9xx.8xxx AKQ8xx.QJ.Axx.xx 0.26982',
 '8xx.T8.KJx.AK9xx Txxx.QJx.9xxx.Tx AKQJ9.Kx.ATx.8xx 0.26436',
 'Txxx.KJ.Tx.AK8xx xx.Txx.K9xxx.9xx AKQJ98.Q8.AJx.Tx 0.26358',
 '8xx.QJx.9.AKT8xx 9x.T8x.AJxxxx.9x AKQJTxx.K.KTx.xx 0.25881',
 'J98.KTx.xx.AK8xx xxx.J8x.JT9xxx.T AKQTxx.Q.AK.9xxx 0.25784',
 '9xxx..KTx.AKT9xx J8x.8xx.J9xxx.xx AKQTx.KQJT.Ax.8x 0.25737',
 'JT9x.Kx.Tx.AKTxx 8xx.J8x.J9xx.8xx AKQxx.QT.AKxx.9x 0.